In [2]:
import glob
import os
import pandas as pd
import numpy as np
import sys
from datetime import datetime, timedelta

SOFA PROCESSING

In [4]:
# read sofa data
file = pd.read_csv('/data/datasets/ICU_Data/EHR_Data/truncated/2020-02-26/sofa_0_trimmed.csv')
file.head(n=20)

,patient_deiden_id,encounter_deiden_id,pain_datetime,pain_pre_post_indicator,pain_uf_dvprs,pain_jax,anes_enc_y_n,intraop_y_n,record_id
0,7412,25797,2019-12-01 10:10:00,pre,0,NaN,N,Y,110
1,7412,25797,2019-12-05 23:30:00,NaN,4 (Mild),NaN,N,Y,110
2,46,86,2018-09-19 07:30:00,pre,NaN,NaN,N,Y,75
3,50,55,2017-10-24 18:00:00,NaN,Patient Asleep,NaN,N,Y,29
4,13,13,2018-08-29 10:03:00,NaN,0,NaN,N,Y,73
5,1,1,2018-07-24 15:00:00,NaN,Patient Asleep,NaN,N,Y,63
6,7403,23648,2019-10-04 11:09:00,NaN,7 (Severe),NaN,N,Y,101
7,71,230,2016-07-15 15:10:00,post,0,NaN,N,Y,4
8,6,6,2018-10-12 11:44:00,NaN,8 (Severe),NaN,N,Y,88
9,60,73,2018-05-01 13:38:00,NaN,Patient Asleep,NaN,N,Y,47


In [3]:
#process sofa data, getting the patient id, timestamp and sofa score

sofa_map = {}
sofa_final = []
for row in file.itertuples():
    #todo: check if the patient id is in the map
    if row.date_of_care in sofa_map and row.record_id == sofa_map[row.patient_id]:
        sofa_map[row.date_of_care]['sofa_score'].append(row.sofa_score)
    else:
        sofa_map[row.date_of_care] = {'sofa_score':[row.sofa_score], 'patient_id':row.record_id}

for timestamp in sofa_map.keys():
    try:
        mean_sofa = np.mean(sofa_map[timestamp]['sofa_score'])
        patient_id = sofa_map[timestamp]['patient_id']
        sofa_final.append([timestamp,patient_id,mean_sofa])
    except:
        print(timestamp)
        print(sofa_map[timestamp]['sofa_score'])
        print(sofa_map[timestamp]['patient_id'])

In [6]:
# save sofa data
np.savez('/home/jsenadesouza/DA-healthy2patient/data/clinical_data/sofa_final.npz', sofa=sofa_final)

In [4]:
print(sofa_final)

[['2016-02-04', 1, 5.0], ['2016-02-03', 1, 5.0], ['2016-02-06', 1, 6.0], ['2016-02-05', 1, 7.5], ['2016-01-31', 1, 4.0], ['2016-01-30', 1, 4.0], ['2016-02-02', 1, 6.0], ['2016-02-01', 1, 6.0], ['2016-02-11', 3, 4.5], ['2016-02-10', 3, 4.0], ['2016-02-13', 3, 4.5], ['2016-02-14', 3, 4.0], ['2016-02-12', 3, 5.0], ['2016-02-09', 3, 4.5], ['2016-02-07', 3, 4.0], ['2016-02-08', 3, 5.0], ['2016-02-19', 3, 4.0], ['2016-02-18', 3, 5.0], ['2016-02-20', 3, 5.0], ['2016-02-15', 3, 5.0], ['2016-02-17', 3, 5.0], ['2016-02-16', 3, 6.0], ['2016-08-19', 5, 2.5], ['2016-08-18', 5, 3.0], ['2016-08-16', 5, 3.5], ['2016-08-17', 5, 3.0], ['2016-08-12', 4, 5.0], ['2016-08-14', 4, 3.0], ['2016-08-13', 4, 5.0], ['2016-08-09', 4, 5.0], ['2016-08-08', 4, 5.0], ['2016-08-11', 4, 6.0], ['2016-08-10', 4, 6.0], ['2016-08-21', 4, 4.0], ['2016-08-20', 4, 5.0], ['2016-08-23', 4, 4.0], ['2016-08-22', 4, 4.0], ['2016-08-15', 4, 3.0], ['2016-07-29', 4, 6.0], ['2016-07-31', 4, 6.0], ['2016-07-30', 4, 7.0], ['2016-07-26', 

Pain data process

In [3]:
# read 2016-2019 pain data
file = pd.read_csv('/data/datasets/ICU_Data/EHR_Data/truncated/2020-02-26/pain_0_trimmed.csv')
file.head()

,patient_deiden_id,encounter_deiden_id,pain_datetime,pain_pre_post_indicator,pain_uf_dvprs,pain_jax,anes_enc_y_n,intraop_y_n,record_id
0,7412,25797,2019-12-01 10:10:00,pre,0,NaN,N,Y,110
1,7412,25797,2019-12-05 23:30:00,NaN,4 (Mild),NaN,N,Y,110
2,46,86,2018-09-19 07:30:00,pre,NaN,NaN,N,Y,75
3,50,55,2017-10-24 18:00:00,NaN,Patient Asleep,NaN,N,Y,29
4,13,13,2018-08-29 10:03:00,NaN,0,NaN,N,Y,73


In [10]:
def process_dvprs(value):
    try:
        score = int(value.split(' ')[0])
    except:
        if value == 'Patient Asleep':
            score = -1
        elif np.isnan(value):
            score = -1
        else:
            sys.exit('error in dvprs')

    return score

In [4]:
#process pain data, getting the patient id, timestamp and sofa score

pain_map = {}
pain_final = []
for row in file.itertuples():
    try:
        pain_datetime = datetime.strptime(row.pain_datetime, '%Y/%m/%d %H:%M:%S')
    except:
        pain_datetime = datetime.strptime(row.pain_datetime, '%Y-%m-%d %H:%M:%S')

    pain_datetime = pain_datetime.strftime('%m/%d/%Y %H:%M:%S.%f')

    if pain_datetime in pain_map:
        if row.record_id == pain_map[pain_datetime]['patient_id']:
            print("duplicate in dates")
    else:
        score = process_dvprs(row.pain_uf_dvprs)
        if score != -1:
            pain_map[pain_datetime] = {'pain_uf_dvprs':score, 'patient_id':row.record_id}

for timestamp in pain_map.keys():
    try:
        mean_pain = pain_map[timestamp]['pain_uf_dvprs']
        patient_id = pain_map[timestamp]['patient_id']
        pain_final.append([timestamp,patient_id,mean_pain])
    except:
        print(timestamp)
        print(pain_map[timestamp]['pain_uf_dvprs'])
        print(pain_map[timestamp]['patient_id'])

In [5]:
# save sofa data
np.savez('/home/jsenadesouza/DA-healthy2patient/data/clinical_data/pain_data_2016-2019.npz', pain=pain_final)

read 2020-2022 pain data

In [11]:
# create a map between the subject_deiden_id and the patient id
patient_map = {}
patient_enrollment = pd.read_excel('/data/daily_data/enrollment_log.xlsx')

for row in patient_enrollment.itertuples():
    patient_map[row.subject_deiden_id] = row._6
    #print(row)
print(patient_map)

{'ICU_2': 60, 'ICU_3': 61, 'ICU_1': 62, 'ICU_7': 63, 'ICU_5': 64, 'ICU_4': 65, 'ICU_6': 66, 'ICU_11': 67, 'ICU_12': 68, 'ICU_9': 69, 'ICU_10': 70, 'ICU_13': 71, 'ICU_14': 3, 'ICU_15': 4, 'ICU_16': 5, 'ICU_17': 6, 'ICU_18': 7, 'ICU_19': 8, 'ICU_20': 9, 'ICU_21': 10, 'ICU_22': 11, 'ICU_23': 12, 'ICU_24': 13, 'ICU_25': 14, 'ICU_26': 15, 'ICU_27': 16, 'ICU_28': 17, 'ICU_29': 19, 'ICU_30': 20, 'ICU_31': 21, 'ICU_32': 22, 'ICU_33': 23, 'ICU_34': 24, 'ICU_35': 25, 'ICU_36': 26, 'ICU_37': 27, 'ICU_38': 28, 'ICU_39': 29, 'ICU_40': 30, 'ICU_41': 31, 'ICU_42': 32, 'ICU_43': 34, 'ICU_44': 35, 'ICU_45': 36, 'ICU_46': 38, 'ICU_47': 39, 'ICU_48': 40, 'ICU_49': 41, 'ICU_50': 42, 'ICU_51': 43, 'ICU_52': 44, 'ICU_53': 45, 'ICU_54': 46, 'ICU_55': 47, 'ICU_56': 48, 'ICU_57': 49, 'ICU_58': 50, 'ICU_59': 51, 'ICU_60': 52, 'ICU_61': 53, 'ICU_62': 54, 'ICU_63': 55, 'ICU_64': 56, 'ICU_65': 58, 'ICU_66': 59, 'ICU_67': 72, 'ICU_68': 73, 'ICU_69': 74, 'ICU_70': 79, 'ICU_71': 80, 'ICU_72': 81, 'ICU_73': 82, 'ICU_7

In [13]:
pain_map = {}
pain_final = []

#get all files
files = glob.glob('/data/daily_data/*/pain*.csv',
                       recursive = True)

pd.read_csv(files[0]).head()

,pain_datetime,measurement_name,measurement_value,patient_deiden_id,encounter_deiden_id
0,2021-07-11 09:15:00,pain_jax,7,ICU_6,ICU_6_11
1,2020-05-09 12:59:00,pain_jax,8,ICU_6,ICU_6_12
2,2020-06-30 15:30:00,pain_jax,0,ICU_5,ICU_5_6
3,2020-06-29 06:39:00,pain_jax,6,ICU_5,ICU_5_6
4,2020-05-10 13:59:00,pain_jax,7,ICU_6,ICU_6_12


In [19]:
for file in files:
    print(file)

/data/daily_data/2021-12-08/pain_0.csv
/data/daily_data/2021-12-13/pain_0.csv
/data/daily_data/2021-12-15/pain_0.csv
/data/daily_data/2021-12-14/pain_0.csv
/data/daily_data/2021-12-16/pain_0.csv
/data/daily_data/2021-12-17/pain_0.csv
/data/daily_data/2021-12-18/pain_0.csv
/data/daily_data/2021-12-21/pain_0.csv
/data/daily_data/2021-12-22/pain_0.csv
/data/daily_data/2022-01-19/pain_0.csv
/data/daily_data/2022-01-20/pain_0.csv
/data/daily_data/2022-01-21/pain_0.csv
/data/daily_data/2022-01-22/pain_0.csv
/data/daily_data/2022-02-01/pain_2.csv
/data/daily_data/2022-02-01/pain_4.csv
/data/daily_data/2022-02-01/pain_5.csv
/data/daily_data/2022-02-01/pain_1.csv
/data/daily_data/2022-02-01/pain_0.csv
/data/daily_data/2022-02-01/pain_16.csv
/data/daily_data/2022-02-01/pain_17.csv
/data/daily_data/2022-02-01/pain_15.csv
/data/daily_data/2022-02-01/pain_18.csv
/data/daily_data/2022-02-01/pain_20.csv
/data/daily_data/2022-02-25/pain_0.csv
/data/daily_data/2022-02-26/pain_0.csv
/data/daily_data/202

In [26]:

for file in files:
    try:
        df = pd.read_csv(file)
        file_filtered = df[df["measurement_name"] == "pain_uf_dvprs"]
    except:
        col_names = ['pain_datetime', 'measurement_name', 'measurement_value', 'patient_deiden_id', 'encounter_deiden_id']
        df = pd.read_csv(file, names=col_names, header=None)
        file_filtered = df[df["measurement_name"] == "pain_uf_dvprs"]

    for row in file_filtered.itertuples():
        try:
            pain_datetime = datetime.strptime(row.pain_datetime, '%Y/%m/%d %H:%M:%S')
        except:
            pain_datetime = datetime.strptime(row.pain_datetime, '%Y-%m-%d %H:%M:%S')

        pain_datetime = pain_datetime.strftime('%m/%d/%Y %H:%M:%S.%f')

        if pain_datetime in pain_map:
            if row.patient_deiden_id == pain_map[pain_datetime]['patient_id']:
                print("duplicate in dates")
        else:
            score = process_dvprs(row.measurement_value)
            if score != -1:
                try:
                    pain_map[pain_datetime] = {'pain_uf_dvprs':score, 'patient_id':patient_map[row.patient_deiden_id]}
                except KeyError as e:
                    #print(e)
                    pass

    for timestamp in pain_map.keys():
        try:
            mean_pain = pain_map[timestamp]['pain_uf_dvprs']
            patient_id = pain_map[timestamp]['patient_id']
            pain_final.append([timestamp,patient_id,mean_pain])
        except:
            print(timestamp)
            print(pain_map[timestamp]['pain_uf_dvprs'])
            print(pain_map[timestamp]['patient_id'])

In [28]:
len(pain_final)

2467864

In [29]:
# save sofa data
np.savez('/home/jsenadesouza/DA-healthy2patient/data/clinical_data/pain_data_2020-2022.npz', pain=pain_final)

Calculate the mean time between measurements

In [31]:
a = sorted(pain_map.keys())
subs = []
for i in range(1, len(a)):
    try:
        data1 = datetime.strptime(a[i-1], '%Y/%m/%d %H:%M:%S.%f')
        data2 = datetime.strptime(a[i], '%Y/%m/%d %H:%M:%S.%f')
    except:
        data1 = datetime.strptime(a[i-1], '%Y-%m-%d %H:%M:%S')
        data2 = datetime.strptime(a[i], '%Y-%m-%d %H:%M:%S')
    sub = (data2-data1).total_seconds()
    subs.append(sub)

print(timedelta(seconds=np.mean(subs)))


ValueError: time data '01/01/2021 00:00:00.000000' does not match format '%Y-%m-%d %H:%M:%S'